In [2]:
import ipywidgets as widgets
from IPython.display import display
from datetime import datetime
import os

class WebCalculator:
    def __init__(self):
        # Создаем файл для истории
        self.history_file = "calculations_history.txt"
        if not os.path.exists(self.history_file):
            with open(self.history_file, 'w', encoding='utf-8') as f:
                f.write("История вычислений:\n")

        # Переменные
        self.current_input = ""
        self.operator = ""
        self.first_number = ""
        self.waiting_for_second_number = False

        # Создаем виджеты
        self.create_widgets()

    def create_widgets(self):
        """Создание веб-интерфейса калькулятора"""
        # Дисплей
        self.display = widgets.Text(
            value='',
            placeholder='0',
            description='',
            disabled=True,
            layout=widgets.Layout(width='300px', height='60px'),
            style={'description_width': 'initial'}
        )

        # Кнопки цифр
        self.buttons = []
        for i in range(10):
            btn = widgets.Button(
                description=str(i),
                layout=widgets.Layout(width='60px', height='60px'),
                style={'button_color': 'lightgray'}
            )
            btn.on_click(self.on_digit_click)
            self.buttons.append(btn)

        # Кнопки операций
        self.btn_add = widgets.Button(
            description='+',
            layout=widgets.Layout(width='60px', height='60px'),
            style={'button_color': 'orange'}
        )
        self.btn_sub = widgets.Button(
            description='-',
            layout=widgets.Layout(width='60px', height='60px'),
            style={'button_color': 'orange'}
        )
        self.btn_mul = widgets.Button(
            description='×',
            layout=widgets.Layout(width='60px', height='60px'),
            style={'button_color': 'orange'}
        )
        self.btn_div = widgets.Button(
            description='÷',
            layout=widgets.Layout(width='60px', height='60px'),
            style={'button_color': 'orange'}
        )

        # Специальные кнопки
        self.btn_equals = widgets.Button(
            description='=',
            layout=widgets.Layout(width='60px', height='60px'),
            style={'button_color': 'green'}
        )
        self.btn_clear = widgets.Button(
            description='C',
            layout=widgets.Layout(width='60px', height='60px'),
            style={'button_color': 'red'}
        )
        self.btn_dot = widgets.Button(
            description='.',
            layout=widgets.Layout(width='60px', height='60px'),
            style={'button_color': 'lightgray'}
        )
        self.btn_backspace = widgets.Button(
            description='⌫',
            layout=widgets.Layout(width='60px', height='60px'),
            style={'button_color': 'lightgray'}
        )

        # Назначаем обработчики
        self.btn_add.on_click(self.on_operator_click)
        self.btn_sub.on_click(self.on_operator_click)
        self.btn_mul.on_click(self.on_operator_click)
        self.btn_div.on_click(self.on_operator_click)
        self.btn_equals.on_click(self.on_equals_click)
        self.btn_clear.on_click(self.on_clear_click)
        self.btn_dot.on_click(self.on_dot_click)
        self.btn_backspace.on_click(self.on_backspace_click)

        # Создаем сетку кнопок
        grid = widgets.GridspecLayout(5, 4, width='300px', height='400px')

        # Дисплей
        grid[0, :] = self.display

        # Первый ряд
        grid[1, 0] = self.buttons[7]
        grid[1, 1] = self.buttons[8]
        grid[1, 2] = self.buttons[9]
        grid[1, 3] = self.btn_div

        # Второй ряд
        grid[2, 0] = self.buttons[4]
        grid[2, 1] = self.buttons[5]
        grid[2, 2] = self.buttons[6]
        grid[2, 3] = self.btn_mul

        # Третий ряд
        grid[3, 0] = self.buttons[1]
        grid[3, 1] = self.buttons[2]
        grid[3, 2] = self.buttons[3]
        grid[3, 3] = self.btn_sub

        # Четвертый ряд
        grid[4, 0] = self.buttons[0]
        grid[4, 1] = self.btn_dot
        grid[4, 2] = self.btn_equals
        grid[4, 3] = self.btn_add

        # Панель дополнительных кнопок
        extra_buttons = widgets.HBox([
            self.btn_clear,
            self.btn_backspace
        ])

        # Отображаем интерфейс
        display(widgets.VBox([
            widgets.Label(value='Калькулятор с историей'),
            grid,
            extra_buttons,
            widgets.Output()  # Для вывода информации
        ]))

        self.output_area = widgets.Output()
        display(self.output_area)

    def on_digit_click(self, btn):
        """Обработка нажатия цифр"""
        if self.waiting_for_second_number:
            self.display.value = ""
            self.waiting_for_second_number = False

        if self.display.value == '0':
            self.display.value = btn.description
        else:
            self.display.value += btn.description

    def on_dot_click(self, btn):
        """Обработка нажатия точки"""
        if '.' not in self.display.value:
            if self.display.value == '':
                self.display.value = '0.'
            else:
                self.display.value += '.'

    def on_operator_click(self, btn):
        """Обработка нажатия операторов"""
        if self.display.value == '':
            return

        self.first_number = self.display.value
        self.operator = btn.description
        self.waiting_for_second_number = True

    def on_equals_click(self, btn):
        """Выполнение вычислений"""
        if not self.first_number or not self.operator:
            return

        second_number = self.display.value
        if not second_number:
            return

        try:
            # Преобразуем символы операторов
            operator_map = {
                '+': '+',
                '-': '-',
                '×': '*',
                '÷': '/'
            }

            op = operator_map[self.operator]
            num1 = float(self.first_number)
            num2 = float(second_number)

            # Выполняем операцию
            if op == '+':
                result = num1 + num2
            elif op == '-':
                result = num1 - num2
            elif op == '*':
                result = num1 * num2
            elif op == '/':
                if num2 == 0:
                    self.display.value = "Ошибка"
                    return
                result = num1 / num2

            # Форматируем результат
            if result.is_integer():
                result_str = str(int(result))
            else:
                result_str = str(round(result, 10)).rstrip('0').rstrip('.')

            # Формируем выражение для отображения
            expression = f"{self.first_number} {self.operator} {second_number}"

            # Отображаем результат
            self.display.value = result_str

            # Сохраняем в файл
            self.save_to_history(expression, result_str)

            # Сбрасываем состояние для следующей операции
            self.first_number = result_str
            self.waiting_for_second_number = True

            # Выводим информацию о сохранении
            with self.output_area:
                print(f"Операция сохранена: {expression} = {result_str}")

        except Exception as e:
            self.display.value = "Ошибка"

    def save_to_history(self, expression, result):
        """Сохранение операции в файл истории"""
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        history_entry = f"[{timestamp}] {expression} = {result}\n"

        with open(self.history_file, 'a', encoding='utf-8') as f:
            f.write(history_entry)

    def on_clear_click(self, btn):
        """Очистка дисплея"""
        self.display.value = ""
        self.first_number = ""
        self.operator = ""
        self.waiting_for_second_number = False

    def on_backspace_click(self, btn):
        """Удаление последнего символа"""
        if self.display.value:
            self.display.value = self.display.value[:-1]

# Создаем и запускаем калькулятор
calc = WebCalculator()

Output()